In [ ]:
import numpy as np
import pandas as pd

In [ ]:
base = pd.read_csv('PFTIT597 - 2025-04-01.CSV', sep=';', encoding='latin1')
base.drop(index=0, inplace=True)
base.columns = base.columns.str.strip().str.replace(' ', '', regex=False)
base.reset_index(drop=True, inplace=True)
base

#SX
SX = pd.read_fwf('SX033125.001', widths=[7, 13, 12, 19, 8, 8, 10, 7, 4, 30, 11, 27, 31, 83], header=None)
SX.drop(index=0, inplace=True)
SX.drop(columns=0, inplace=True)
SX.columns = ['A', 'id', 'B', 'C', 'fecha', 'D', 'E', 'F', 'G', 'importe', 'H', 'I', 'J']

# SW - Cod. Bolsa
SW = pd.read_fwf('SW033125.002', widths=[18, 12, 18], header=None)
SW.drop(index=0, inplace=True)
SW.columns = ['id', 'fecha', 'importe']
SW['id'] = SW['id'].str[7:]
SW['importe'] = SW['importe'].str[6:]
SW['importe'] = SW['importe'].astype(str).str.lstrip('0').replace('', '0').astype(float)

# MX - PRECIO SUCIO
MX = pd.read_csv('MX033125.txt', sep=',', encoding='latin1')

# titulos_participativos_valoracion
titulos = pd.read_fwf('titulos_participativos_valoracion_20250331.txt', widths=[19, 12, 20], header=None)
titulos.columns = ['id', 'fecha', 'importe']
titulos['importe'] = titulos['importe'].apply(lambda x: f"{x:.6f}")
titulos['id'] = titulos['id'].str[7:]

In [ ]:
# SW
base['CHECK_SW COD BOLSA'] = pd.to_numeric(base['Cod.Bolsa'].map(SW.set_index('id')['importe']), errors='coerce').fillna(0)

# MX - TITULOS - SX
base['CHECK ORIGEN'] = np.nan
base['CHECK ISIN'] = np.nan

base.loc[base['ISIN'].isin(MX['ISIN']), 'CHECK ORIGEN'] = 'MX'
base.loc[base['ISIN'].isin(titulos['id']), 'CHECK ORIGEN'] = base['CHECK ORIGEN'].fillna('') + ' titulos'
base.loc[base['ISIN'].isin(SX['id']), 'CHECK ORIGEN'] = base['CHECK ORIGEN'].fillna('') + ' SX'

base['CHECK ORIGEN'] = base['CHECK ORIGEN'].str.strip()
base['CHECK ISIN'] = pd.Series(dtype='float')

mx_map = MX.set_index('ISIN')['Precio Sucio']
base.loc[base['ISIN'].isin(mx_map.index), 'CHECK ISIN'] = pd.to_numeric(base['ISIN'].map(mx_map), errors='coerce')

titulos_map = titulos.set_index('id')['importe']
base.loc[base['CHECK ISIN'].isna() & base['ISIN'].isin(titulos_map.index), 'CHECK ISIN'] = pd.to_numeric(base['ISIN'].map(titulos_map), errors='coerce')

sx_map = SX.set_index('id')['importe']
base.loc[base['CHECK ISIN'].isna() & base['ISIN'].isin(sx_map.index), 'CHECK ISIN'] = pd.to_numeric(base['ISIN'].map(sx_map), errors='coerce')

base['CHECK ISIN'] = base['CHECK ISIN'].fillna(0)

In [ ]:
# CHECK DE ARCHIVO DE ORIGEN
origen_codigos = 'SX'
base[base['CHECK ORIGEN'] == origen_codigos]['ISIN'].dropna().unique()